# Monte Carlo Simulation 

The simulations can model the different outcomes (finding the probability distribution of an experiment using random sampling) of a random process if they had taken a different path or sequence.

A Monte Carlo simulation takes the variable that has uncertainty and assigns it a random value. The model is then run and a result is provided. This process is repeated again and again while assigning many different values to the variable in question. 

1. Testing for different values of factors that influence asset prices, we could obtain a good idea about the possible prices of the asset using a function that represents the development of the asset price.

2. By Monte Carlo simulation in trading you get a better understanding of the risk and uncertainty of your trading strategy because Monte Carlo simulation lets you run your backtest thousands of times in different orders thus generating multiple equity curves.

It simply shows how liable to chance and randomness your trading strategy is. What might the future bring when you start trading live? The Monte Carlo simulation takes your trades and reshuffles the trade order to check different paths.

## Roulette Wheel

In [13]:
import random

class FairRoulette():
    
	def __init__(self):
		self.pockets = []
		for i in range(1,37):
			self.pockets.append(i)
		self.ball = None
        # A single bet on a number in roulette is a 'straight up' bet, and it offers high payout 35:1.
		self.pocketOdds = len(self.pockets) - 1
        
	def spin(self):
		self.ball = random.choice(self.pockets)
        
	def betPocket(self, pocket, amt):
		if str(pocket) == str(self.ball):
			return amt*self.pocketOdds
		else: return -amt

	def __str__(self):
		return 'Fair Roulette'

def playRoulette(game, numSpins, pocket, bet, toPrint):
    totPocket = 0
    for i in range(numSpins):
        game.spin()
        totPocket += game.betPocket(pocket, bet)
    if toPrint:
        print(numSpins, 'spins of', game)
        print('Expected return betting', pocket, '=',\
            str(100*totPocket/numSpins) + '%\n')
    return (totPocket/numSpins)

class EuRoulette(FairRoulette):

	def __init__(self):
		FairRoulette.__init__(self)
		self.pockets.append('0')

	def __str__(self):
		return 'European Roulette'

class AmRoulette(EuRoulette):

	def __init__(self):
		EuRoulette.__init__(self)
		self.pockets.append('00')

	def __str__(self):
		return 'American Roulette'

# game = AmRoulette()
# for numSpins in (10, 100000):
# 	for i in range(3):
# 		playRoulette(game, numSpins, 2, 1, True)

def findPocketReturn(game, numTrials, trialSize, toPrint):
    pocketReturns = []
    for t in range(numTrials):
        trialVals = playRoulette(game, trialSize, 2, 1, toPrint)
        pocketReturns.append(trialVals)
    return pocketReturns

random.seed(0)
numTrials = 20
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette)
for G in games:
    resultDict[G().__str__()] = []
for numSpins in (1000, 10000, 100000, 1000000):
    print('\nSimulate', numTrials, 'trials of',
          numSpins, 'spins each')
    for G in games:
        pocketReturns = findPocketReturn(G(), numTrials,
                                         numSpins, False)
        expReturn = 100*sum(pocketReturns)/len(pocketReturns)
        print('Exp. return for', G(), '=',
             str(round(expReturn, 4)) + '%')
             
def getMeanAndStd(X):
    mean = sum(X)/float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    std = (tot/len(X))**0.5
    return mean, std



Simulate 20 trials of 1000 spins each
Exp. return for Fair Roulette = 6.56%
Exp. return for European Roulette = -2.26%
Exp. return for American Roulette = -8.92%

Simulate 20 trials of 10000 spins each
Exp. return for Fair Roulette = -1.234%
Exp. return for European Roulette = -4.168%
Exp. return for American Roulette = -5.752%

Simulate 20 trials of 100000 spins each
Exp. return for Fair Roulette = 0.8144%
Exp. return for European Roulette = -2.6506%
Exp. return for American Roulette = -5.113%

Simulate 20 trials of 1000000 spins each
Exp. return for Fair Roulette = -0.0723%
Exp. return for European Roulette = -2.7329%
Exp. return for American Roulette = -5.212%


### Gambler's Fallacy

"On August 18, 1913, at the casino in Monte Carlo, black came up a record twenty-six times in succession [in roulette]. [There] was a near-panicky rush to bet on red, beginning about the time black had come up a phenomenal fifteen times." -- Huff and Geis, *How to Take a Chance*

~Probability of 26 consecutive reds: 1/67,108,645. \
~Probability of 26 consecutive reds when previous 25 rolls were red is 1/2.

**Monte Carlo fallacy**: If an event has occurred more frequently than expected, it is less likely to happen again in the future.

### Regression to the mean

Coined in 1885 by Francis Galt in his paper summarized by: If somebody's parents are both taller than average, it's likely that the child will be smaller than the parents. Conversely, if the parents are shorter than average, it's likely that the child will be taller than average.

"Following an extreme random event, the next random event is likely to be less extreme." \
If you spin a fair roulette wheel 10 times and get 100% reds, that is an extreme event (probability = 1/1024). lt is likely that in the next 10 spins, you will get fewer than 10 reds, but the expected number is 5.

## Simple example of Buffon-Laplace Method

In [9]:
import random
import scipy.stats as stats

def estimate_pi(num_points):
    points_inside_circle = 0
    points_total = 0

    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        distance = x**2 + y**2

        if distance <= 1:
            points_inside_circle += 1

        points_total += 1

    pi_estimate = 4 * (points_inside_circle / points_total)
    return pi_estimate

def getEst(num_points, numTrials): 
    estimates = [] 
    for t in range(numTrials): 
        piGuess = estimate_pi(num_points) 
        estimates.append(piGuess) 
    sDev = stats.tstd(estimates) 
    curEst = sum(estimates)/len(estimates) 
    print('Est. = ' + str(curEst) +  
          ', Std. dev. = ' + str(round(sDev, 6)) + 
          ', Points = ' + str(num_points)) 
    return (curEst, sDev)

def estPi(precision, numTrials): 
    num_points = 1000 
    sDev = precision 
    while sDev >= precision: 
        curEst, sDev = getEst(num_points, numTrials) 
        num_points *= 2 
    # Print the estimated value of π
    print("Estimated value of π:", curEst, "with precision:",precision) 

# Run the Monte Carlo simulation
numTrials = 100
precision = 0.005
estPi(precision, numTrials)

Est. = 3.1516, Std. dev. = 0.058811, Points = 1000
Est. = 3.1376599999999994, Std. dev. = 0.03785, Points = 2000
Est. = 3.1413399999999996, Std. dev. = 0.026066, Points = 4000
Est. = 3.1433300000000015, Std. dev. = 0.017334, Points = 8000
Est. = 3.141525000000002, Std. dev. = 0.012978, Points = 16000
Est. = 3.1425887500000003, Std. dev. = 0.010184, Points = 32000
Est. = 3.143143125000001, Std. dev. = 0.006417, Points = 64000
Est. = 3.1407181250000002, Std. dev. = 0.00445, Points = 128000
Estimated value of π: 3.1407181250000002 with precision: 0.005


## Simulating investment portfolio growth

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def monte_carlo_simulation(principal, annual_return, volatility, num_years, num_simulations):
    investment_values = []

    for _ in range(num_simulations):
        values = [principal]
        for _ in range(num_years):
            growth_rate = np.random.normal(annual_return, volatility)
            investment_value = values[-1] * (1 + growth_rate)
            values.append(investment_value)
            
        investment_values.append(values)

    return investment_values

# Set the simulation parameters
initial_principal = 100000  # Initial investment principal
annual_return = 0.03       # Expected annual return (mean)
volatility = 0.35         # Volatility of returns (standard deviation)
years = 40                # Number of years to simulate
simulations = 100        # Number of simulations

# Run the Monte Carlo simulation
simulation_results = monte_carlo_simulation(initial_principal, annual_return, volatility, years, simulations)

# Get the final values from each simulation
final_values = [values[-1] for values in simulation_results]

# Calculate the mean and standard deviation of the final values
mean_final_value = np.mean(final_values)
std_final_value = np.std(final_values)

# Print the results
print("Mean final value:", mean_final_value)
print("Standard deviation of final value:", std_final_value)

# Plot the investment values for each simulation
plt.figure(figsize=(10, 6))
for values in simulation_results:
    plt.plot(range(years + 1), values) #[value / initial_principal for value in values]

plt.xlabel('Years')
plt.ylabel('Investment Value')
plt.title('Monte Carlo Simulation - Investment Portfolio')
plt.grid(True)
plt.show()

# Define the bust threshold
bust_threshold = 0

# Check which investments went bust
bust_investments = []
for i, values in enumerate(simulation_results):
    final_value = values[-1]
    if final_value <= bust_threshold:
        bust_investments.append(i)

# Print the investments that went bust
print("Investments that went bust:", bust_investments)

## Simulating stock prices using Geometric Brownian Motion and the Monte Carlo method

### Black-Scholes financial asset model

An asset's price movement consists of two main factors: drift, which represents its consistent directional change, and a random component, which captures market volatility that computed based on historical price data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

In [ ]:
daily_asset_adjclose = pd.DataFrame(yf.download('JPM')['Adj Close'])

In [ ]:
log_returns = np.log(1+daily_asset_adjclose.pct_change())

Obtain the average daily return, variance and standard deviation to calculate drift = mean - variance*0.5

In [ ]:
mean = log_returns.mean()
variance = log_returns.var()
standard_deviation = log_returns.std()

In [ ]:
drift = mean - variance/2

In [ ]:
def monte_carlo_simulation(adjclose, drift, stdev, days, trials):
    normal_rv = norm.ppf(np.random.rand(days, trials))
    daily_returns = np.exp(np.array(drift) + (np.array(stdev) * normal_rv))
    price_paths = np.zeros_like(daily_returns)
    
    price_paths[0] = adjclose.iloc[-1, 0]
    for t in range(1, days):
        price_paths[t] = price_paths[t-1] * daily_returns[t]

    return price_paths

In [ ]:
# Set the parameters
days = 40
trials = 150

# Run the Monte Carlo simulation
simulation_results = monte_carlo_simulation(daily_asset_adjclose, drift, standard_deviation, days, trials)

plt.figure(figsize=(12,8))
plt.plot(simulation_results)
plt.xlabel('Days')
plt.ylabel('Price Action Paths')
plt.title('Monte Carlo Simulation - Geometric Brownian Motion')
plt.grid(True)
plt.show()